# MIDI LSTM Music Generator
This houses the code for training an LSTM neural network on a text-based intermediate representation and creating a model which can predict (or "compose") music based on the learned weights and biases.

In [3]:
def wrapper():
    import os
    import random
    import sys
    import time
    import numpy as np
    import tensorflow as tf

    from tensorflow.contrib import rnn
    from hops import tensorboard
    
    # Filesystem setup
    from hops import hdfs
    fs_handle = hdfs.get_fs()
    
    # Parsing constants
    PROJECT_DIR = hdfs.project_path()
    SAMPLE_DIR = PROJECT_DIR + "ParsedMIDI/Sample"
    CHORD_DELIM = ' '
    
    # TODO Make directory with /**/**.txt
    input_file = SAMPLE_DIR + "/beethoven_hammerklavier_1_format0_track[0].txt"
    save_loc = PROJECT_DIR + "SavedModels/model.ckpt"
    
    hdfs.log("input_file: {}".format(input_file))
    hdfs.log("save_loc: {}".format(save_loc))
    
    """ HELPER FUNCTIONS """
    def variable_summaries(var):
        """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
        with tf.name_scope('summaries'):
            mean = tf.reduce_mean(var)
            tf.summary.scalar('mean', mean)
            with tf.name_scope('stddev'):
                stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
            tf.summary.scalar('stddev', stddev)
            tf.summary.scalar('max', tf.reduce_max(var))
            tf.summary.scalar('min', tf.reduce_min(var))
            tf.summary.histogram('histogram', var)

    def build_dicts(words):
        """ TODO Use below, but figure out how to handle distributed/parallel workers and multi-note chords
        d = {}
        rd = {}
        for i in range(0, 127):
            d[i] = chr(i)
            rd[chr(i)] = i
        return (d, rd)
        """
        import collections
        counts = collections.Counter(words).most_common()
        dictionary = dict()
        for word, _count in counts:
            dictionary[word] = len(dictionary)
        reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
        return dictionary, reverse_dictionary


    def read_file(handle, file_path):
        """
        Consumes a given textfile by stripping and splitting it into words
        :param handle: HDFS handle
        :param file_path: location of file to read
        :return: numpy array of the file's "words"
        """
        with fs_handle.open_file(file_path, "r") as f:
            lines = []
            while True:
                try:
                    lines.append(f.next())
                except StopIteration:
                    break
        content = [lines[i].split(CHORD_DELIM) for i in range(len(lines))]
        content = np.array(content)
        content = np.reshape(content, [-1, ])
        return content
    
    def read_data(handle, data_path):
        # TODO Handle directories!
        return [read_file(handle, data_path)]
    
    def get_lstm_cell(num_hidden):
        """
        Creates a new LSTM cell
        :param num_hidden: number of units in the cell
        :return a new LSTM cell
        """
        return rnn.BasicLSTMCell(num_hidden)
    
    def create_rnn(x, weights, biases, num_inputs, num_hidden):
        """
        Creates a multi-layer RNN based on LSTM cells
        :param x: inputs
        :param weights: dictionary of weight variables
        :param biases: dictionary of bias variables
        :param num_inputs: # of sequence inputs to the RNN
        :param num_hidden: # of units in the RNN cells
        :return the fully constructed RNN
        """
        
        def make_cell():
            """
            Generates an RNN cell
            :return the fully constructed RNN cell with optional dropoff (TODO)
            """
            cell = get_lstm_cell(num_hidden)
            # TODO if is_training and keep_prob < 1:
            #    cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=config.keep_prob)
            return cell

        x = tf.reshape(x, [-1, num_inputs])

        # Split into n-element sequences of inputs
        x = tf.split(x, num_inputs, 1)

        num_layers = 2

        rnn_cell = rnn.MultiRNNCell([make_cell() for _ in range(num_layers)])

        # Generate prediction
        outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

        # there are num_inputs outputs but
        # we only want the last output
        return tf.matmul(outputs[-1], weights['out']) + biases['out']


    """ MAIN PROGRAM """
    # Parameters
    learning_rate = 0.0001
    training_iters = 5000
    display_step = 1000
    n_input = 5
    n_predictions = 32
    n_hidden = 512
    
    # TODO Consume and concatenate multiple files (see chopin.py in repo)
    training_data = read_data(fs_handle, input_file)
    hdfs.log("training_data: {}".format(training_data))
    
    # Flatten into 1D-array (needed with multiple file inputs)
    training_data = np.concatenate(training_data).ravel()
    
    # Prepare dictionary/vocabulary
    dictionary, reverse_dictionary = build_dicts(training_data)
    vocab_size = len(dictionary)

    
    x = tf.placeholder("float", [None, n_input, 1])
    y = tf.placeholder("float", [None, vocab_size])

    weights = {
        'out': tf.Variable(tf.random_normal([n_hidden, vocab_size], mean=0.0, stddev=0.08))
    }
    # TODO Initialize LSTM forget gates with higher biases to encourage remembering in beginning
    biases = {
        'out': tf.Variable(tf.random_normal([vocab_size], mean=0.0, stddev=0.8))
    }

    pred = create_rnn(x, weights, biases, n_input, n_hidden)

    # Loss and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
    with tf.name_scope('train'):
        with tf.name_scope('adam_optimizer'):
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
            train_op = optimizer.minimize(cost, global_step=tf.train.get_or_create_global_step())

    # Model evaluation
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    # Summaries
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('cost', cost)

    
    
    # Initializing the variables
    init = tf.global_variables_initializer()
    summary_op = tf.summary.merge_all()
    saver = tf.train.Saver()
    
    logdir = tensorboard.logdir()
    writer = tf.summary.FileWriter(logdir=logdir)
    
    with tf.Session() as session:
        session.run(init)
        step = 0
        offset = random.randint(0, n_input + 1)
        end_offset = n_input + 1
        acc_total = 0
        loss_total = 0

        writer.add_graph(session.graph)

        while step < training_iters:
            if offset > (len(training_data) - end_offset):
                # If we've stepped past our input data, restart at random offset
                offset = random.randint(0, n_input + 1)

            symbols_in_keys = [[dictionary[str(training_data[i])]] for i in range(offset, offset + n_input)]
            symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

            symbols_out_onehot = np.zeros([vocab_size], dtype=float)
            symbols_out_onehot[dictionary[str(training_data[offset + n_input])]] = 1.0
            symbols_out_onehot = np.reshape(symbols_out_onehot, [1, -1])

            summary, _, acc, loss, onehot_pred = session.run(
                [summary_op, train_op, accuracy, cost, pred],
                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
            
            writer.add_summary(summary, step)
            
            loss_total += loss
            acc_total += acc
            if (step + 1) % display_step == 0:
                print("Iter= " + str(step + 1) + ", Average Loss= " +
                      "{:.6f}".format(loss_total / display_step) + ", Average Accuracy= " +
                      "{:.2f}%".format(100 * acc_total / display_step))
                acc_total = 0
                loss_total = 0
                symbols_in = [training_data[i] for i in range(offset, offset + n_input)]
                symbols_out = training_data[offset + n_input]
                symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
                print("%s - [%s] vs [%s]" % (symbols_in, symbols_out, symbols_out_pred))
            step += 1
            offset += (n_input + 1)
        print("Optimization Finished!")
        save_path = saver.save(session, save_loc)
        print("Model saved in file: %s" % save_path)
        
    # Cleanup
    fs_handle.close()
    writer.close()


In [ ]:
# Launcher
from hops import tflauncher
from hops import tensorboard

tf_hdfs_dir = tflauncher.launch(spark, wrapper)
tensorboard.visualize(spark, tf_hdfs_dir)